In [1]:
from dgl.data import citation_graph
import torch
import numpy as np

In [36]:
data =citation_graph.CoraDataset()
labels=np.zeros(data.labels.shape)

In [17]:
print(data.train_mask.sum())
print(data.val_mask.sum())
print(data.test_mask.sum())

140.0
300.0
1000.0


In [25]:
data.labels.shape

(2708,)

In [7]:
labels

array([0., 1., 1., ..., 1., 1., 0.])

In [35]:
from collections import Counter
Counter(data.labels)

Counter({1: 2708})

In [29]:
Counter(labels)

Counter({0: 818, 1: 1890})

In [31]:
def one_class_processing(labels,normal_class:int):
    labels,normal_idx,abnormal_idx=one_class_labeling(labels,normal_class)
    
    return one_class_masking(labels,normal_idx,abnormal_idx)


def one_class_labeling(labels,normal_class:int):
    normal_class=normal_class
    normal_idx=np.where(labels==normal_class)[0]
    abnormal_idx=np.where(labels!=normal_class)[0]
    np.random.shuffle(normal_idx)
    np.random.shuffle(abnormal_idx)
    labels[normal_idx]=0
    labels[abnormal_idx]=1

    return labels,normal_idx,abnormal_idx

def one_class_masking(labels,normal_idx,abnormal_idx):
    train_mask=np.zeros(labels.shape)
    val_mask=np.zeros(labels.shape)
    test_mask=np.zeros(labels.shape)

    train_mask[normal_idx[:int(0.6*normal_idx.shape[0])]]=1

    val_mask[normal_idx[int(0.6*normal_idx.shape[0]):int(0.8*normal_idx.shape[0])]]=1
    val_mask[abnormal_idx[:int(0.5*abnormal_idx.shape[0])]]=1

    test_mask[normal_idx[int(0.8*normal_idx.shape[0]):]]=1
    test_mask[abnormal_idx[int(0.5*abnormal_idx.shape[0]):]]=1

    return labels,train_mask,val_mask,test_mask  

In [37]:
labels,train_mask,val_mask,test_mask=one_class_processing(data.labels,2)

In [19]:
train_mask=np.zeros(data.labels.shape)
val_mask=np.zeros(data.labels.shape)
test_mask=np.zeros(data.labels.shape)

In [38]:
print(Counter(train_mask))
print(Counter(val_mask))
print(Counter(test_mask))

Counter({0.0: 2218, 1.0: 490})
Counter({0.0: 1599, 1.0: 1109})
Counter({0.0: 1599, 1.0: 1109})


In [20]:
train_mask[normal_idx[:int(0.6*normal_idx.shape[0])]]=1

In [22]:
val_mask[normal_idx[int(0.6*normal_idx.shape[0]):int(0.8*normal_idx.shape[0])]]=1
val_mask[abnormal_idx[:int(0.5*abnormal_idx.shape[0])]]=1

In [23]:
test_mask[normal_idx[int(0.8*normal_idx.shape[0]):]]=1
test_mask[abnormal_idx[int(0.5*abnormal_idx.shape[0]):]]=1

In [ ]:
train_mask

In [ ]:
data.labels